In [ ]:
# MOUNTING GOOGLE DRIVE
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !python --version

Python 3.9.16


In [ ]:
# import os

# # Install java
# ! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null

# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
# ! java -version

In [ ]:
# INSTALLING PACKAGEs TO READ 5GB JSON FILE
!pip install ijson
# !pip install pyspark
# !pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# import findspark
# findspark.init()

In [ ]:
data_path = "/content/drive/MyDrive/CS 532/cs532_project/data"

##Reading raw JSON file

In [ ]:
import os
import json
import ijson
import pandas as pd

# JSON DATA PATH
json_file_name = os.path.join(data_path, "yelp_academic_dataset_review.json")

# READING JSON PATH USING THE IJSON ITERATOR
counter = 0
data = []
for prefix, the_type, value in ijson.parse(open(json_file_name), multiple_values=True):
    # print(f"prefix = {prefix}, the_type = {the_type}, value = {value}")
    counter += 1

    if the_type == "start_map": item_list = ['']*9
    
    if prefix == "review_id": item_list[0] = value
    if prefix == "user_id": item_list[1] = value
    if prefix == "business_id": item_list[2] = value
    if prefix == "stars": item_list[3] = value
    if prefix == "useful": item_list[4] = value
    if prefix == "funny": item_list[5] = value
    if prefix == "cool": item_list[6] = value
    if prefix == "text": item_list[7] = value
    if prefix == "date": item_list[8] = value

    if the_type == "end_map": data.append(item_list)

    # if counter >=100: break



In [ ]:
len(data) # 6990280

6990280

In [ ]:
csv_file_raw = os.path.join(data_path, "review_raw.csv")
raw_df = pd.DataFrame(data, columns=["review_id", "user_id", "business_id", "stars", 
                                        "useful", "funny", "cool", "text", "date"])
raw_df.to_csv(csv_file_raw, index=False,)
# data_file.close()

In [ ]:
# from pyspark.context import SparkContext
# from pyspark.sql.session import SparkSession

# spark = SparkSession.builder.config("spark.driver.memory", "6g").getOrCreate()
# # spark = SparkSession.builder \
#       #  .master("local[2]") \
#       #  .appName("test") \
#       #  .config("spark.driver.memory", "6g") \
#       #  .getOrCreate()
# sc = spark.sparkContext
# from pyspark.sql import SQLContext
# sqlContext = SQLContext(sc)

# # sc = SparkContext.getOrCreate()
# # spark = SparkSession(sc)

/usr/local/lib/python3.9/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


## Sorting raw CSV file w.r.t date

In [1]:
import os
csv_file_raw = os.path.join(data_path, "review_raw.csv")

NameError: ignored

In [ ]:
import numpy as np
import time
import pandas as pd
  
# time taken to read data
s_time_chunk = time.time()
chunk = pd.read_csv(csv_file_raw, chunksize=1000)
e_time_chunk = time.time()
  
print("With chunks: ", (e_time_chunk-s_time_chunk), "sec")

s_time_chunk = time.time()
df = pd.concat(chunk)
e_time_chunk = time.time()
  
print("With chunks: ", (e_time_chunk-s_time_chunk), "sec")

In [ ]:
df.sort_values(by=['date'], inplace=True)
df = df.reset_index(drop = True)
# raw_df_sorted = spark.read.csv(csv_file_raw, header=True).coalesce(1).orderBy('date').toPandas()
# df.head()

In [ ]:
csv_file_raw_sorted = os.path.join(data_path, "review_raw_sorted.csv")
df.to_csv(csv_file_raw_sorted, index=False)

In [ ]:
df.shape

## Generating sentiment labels and year column in sorted csv

In [ ]:
import numpy as np
import os
import time
import pandas as pd

csv_file_raw_sorted = os.path.join(data_path, "review_raw_sorted_labels_v2.csv")

# time taken to read data
s_time_chunk = time.time()
chunk = pd.read_csv(csv_file_raw_sorted, chunksize=10000)
e_time_chunk = time.time()
  
print("With chunks: ", (e_time_chunk-s_time_chunk), "sec")

s_time_chunk = time.time()
df = pd.concat(chunk)
e_time_chunk = time.time()
  
print("With chunks: ", (e_time_chunk-s_time_chunk), "sec")

With chunks:  0.7743775844573975 sec
With chunks:  135.4234402179718 sec


In [ ]:
def f3(row):
    if row['stars'] >= 4:
        val = 'positive'
    elif row['stars'] == 3:
        val = 'neutral'
    else:
        val = 'negative'
    return val

def f2(row):
    if row['stars'] >= 4:
        val = 'positive'
    else:
        val = 'negative'
    return val

def year(row):
  if isinstance(row['date'], str):
    val = row['date'][:4]
  else:
    val = 'check'
    print(type(row['date']), row['date'])
  return val

# df['labels_3'] = df.apply(f3, axis=1)
# df['labels_2'] = df.apply(f2, axis=1)
df['year'] = df.apply(year, axis=1)

<class 'float'> nan
<class 'float'> nan
<class 'float'> nan


In [ ]:
df.head(10)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,labels_3,labels_2,year
0,IykJMMZgbNcUndwf1q6ieg,3zBJUlWtPNoZ0uN83ODbyg,2bXm0SynOfxDzfrdrCyXqg,4.0,0.0,0.0,0.0,It's not chicago style pizza but it is still a...,2005-02-16 03:23:22,positive,positive,2005
1,-O5ng1XLox6uEr4uIZ8u5A,3zBJUlWtPNoZ0uN83ODbyg,3g6XqkBikTTbZmTukbeGnw,4.0,1.0,0.0,0.0,Great service. The people really want to help...,2005-02-16 03:29:39,positive,positive,2005
2,g80vzN72iU03Wh0fSpq41g,3zBJUlWtPNoZ0uN83ODbyg,PP3BBaVxZLcJU54uP_wL6Q,5.0,0.0,0.0,0.0,These guys really are the king of cheese steak...,2005-02-16 04:06:26,positive,positive,2005
3,7Dcrt0Oz0hikA8obGhadrQ,XCsZ3hWa_6oP1WkWvK7pmg,U3grYFIeu6RgAAQgdriHww,5.0,0.0,0.0,0.0,Words cannot describe how much I miss this pla...,2005-03-01 16:57:17,positive,positive,2005
4,WC9q5vhQlQkLK05kEs-vYQ,XCsZ3hWa_6oP1WkWvK7pmg,Aes-0Q_guDeYewMapFs_vg,2.0,0.0,0.0,0.0,Food is decent but is a complete tourist trap....,2005-03-01 16:59:37,negative,negative,2005
5,Q0GJ06L78nkVyNfHrd9iUg,XCsZ3hWa_6oP1WkWvK7pmg,CziOtnFSklimJnBgksDDwA,3.0,0.0,0.0,0.0,"This place gets a 3.5...great location, great ...",2005-03-01 17:25:13,neutral,negative,2005
6,TYslH-CAecjJxLNs96KduA,XCsZ3hWa_6oP1WkWvK7pmg,29YqJwOGEuAWqlHZxMc1OA,5.0,0.0,0.0,0.0,Lovely little restaurant which was converted f...,2005-03-01 17:47:15,positive,positive,2005
7,b5IWJOAwIKwyOsOGbS-sCA,XCsZ3hWa_6oP1WkWvK7pmg,6RBZfirnzE4NahJTn1UPNA,4.0,0.0,0.0,0.0,This little hole in the wall taqueria is writt...,2005-03-01 17:59:26,positive,positive,2005
8,hX0XEpQ0Ai1UgHEQjrWOkQ,XCsZ3hWa_6oP1WkWvK7pmg,jssubXBNkiSJIhbKuDtmpQ,4.0,0.0,0.0,0.0,This is the original location for this great t...,2005-03-01 19:33:35,positive,positive,2005
9,sg2Mc_sanqnZ__lXKZk3hA,XCsZ3hWa_6oP1WkWvK7pmg,bSfDI69RkUrodsA1QwgnOg,5.0,0.0,0.0,0.0,Oh the Sh*tty Kitty...by far the best bar in S...,2005-03-01 19:35:44,positive,positive,2005


In [ ]:
raw_sorted_wLabels = os.path.join(data_path, "review_raw_sorted_labels_v2.csv")
# df.to_csv(raw_sorted_wLabels, index=False)

In [ ]:
df['year'].value_counts()

2019     907284
2018     906362
2017     820048
2016     758881
2015     688415
2021     618189
2020     554557
2014     522275
2013     383950
2012     286570
2011     230813
2010     138587
2009      74387
2008      48226
2022      31665
2007      15363
2006       3853
2005        854
check         3
Name: year, dtype: int64

In [ ]:
## count of reviews by year

# 2022      31665
# 2021     618189
# 2020     554557
# 2019     907284
# 2018     906362
# 2017     820048
# 2016     758881
# 2015     688415
# 2014     522275
# 2013     383950
# 2012     286570
# 2011     230813
# 2010     138587
# 2009      74387
# 2008      48226
# 2007      15363
# 2006       3853
# 2005        854
# check         3

## Creating partitioned data

In [ ]:
df.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,labels_3,labels_2,year
0,IykJMMZgbNcUndwf1q6ieg,3zBJUlWtPNoZ0uN83ODbyg,2bXm0SynOfxDzfrdrCyXqg,4.0,0.0,0.0,0.0,It's not chicago style pizza but it is still a...,2005-02-16 03:23:22,positive,positive,2005
1,-O5ng1XLox6uEr4uIZ8u5A,3zBJUlWtPNoZ0uN83ODbyg,3g6XqkBikTTbZmTukbeGnw,4.0,1.0,0.0,0.0,Great service. The people really want to help...,2005-02-16 03:29:39,positive,positive,2005
2,g80vzN72iU03Wh0fSpq41g,3zBJUlWtPNoZ0uN83ODbyg,PP3BBaVxZLcJU54uP_wL6Q,5.0,0.0,0.0,0.0,These guys really are the king of cheese steak...,2005-02-16 04:06:26,positive,positive,2005
3,7Dcrt0Oz0hikA8obGhadrQ,XCsZ3hWa_6oP1WkWvK7pmg,U3grYFIeu6RgAAQgdriHww,5.0,0.0,0.0,0.0,Words cannot describe how much I miss this pla...,2005-03-01 16:57:17,positive,positive,2005
4,WC9q5vhQlQkLK05kEs-vYQ,XCsZ3hWa_6oP1WkWvK7pmg,Aes-0Q_guDeYewMapFs_vg,2.0,0.0,0.0,0.0,Food is decent but is a complete tourist trap....,2005-03-01 16:59:37,negative,negative,2005


In [ ]:
df_2005.columns

Index(['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny',
       'cool', 'text', 'date', 'labels_3', 'labels_2', 'year'],
      dtype='object')

In [ ]:
df_2008 = df_2008.drop(['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny','cool', 'date', 'labels_2', 'year', ], axis=1)

In [ ]:
df.iloc[0]['year'] == '2005'

False

In [ ]:
df_2005 = df.loc[(df['year'] == 2005) | (df['year'] == 2006) | (df['year'] == 2007)]
df_2008 = df.loc[(df['year'] == 2008)]

In [ ]:
df.shape, df_2005.shape, df_2008.shape

((6990282, 12), (20070, 12), (48226, 2))

In [ ]:
df_2008.to_csv(os.path.join(data_path, "review_2008.csv"), index=False)

In [ ]:
df_2005.head()

,text,labels_3
0,It's not chicago style pizza but it is still a...,positive
1,Great service. The people really want to help...,positive
2,These guys really are the king of cheese steak...,positive
3,Words cannot describe how much I miss this pla...,positive
4,Food is decent but is a complete tourist trap....,negative


In [ ]:
df_2005['labels_3'].value_counts()

positive    599
neutral     171
negative     84
Name: labels_3, dtype: int64